In [1]:
from __future__ import division, print_function, absolute_import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
ethnea_df = pd.read_csv('names_ethnea_genni_country.csv')
#ethnea_df

In [3]:
# First thing first, use the character feature for making the dnn model
def extract_structure(word,n_char=2):
    x_struct = []
    word_len = len(word) + n_char
    n_char-=1
    counter = 0
    for i in range(word_len):
        end = i+1
        start = (i - n_char) if (i - n_char) > 0 else 0
        if word[start:end]!='_' and word[start:end]!='':
        #if word[start:end]!='_':
            x_struct.append(word[start:end])
    return x_struct

first_name_struct = ethnea_df.First.apply(lambda x: extract_structure(x.lower(),2))
last_name_struct = ethnea_df.Last.apply(lambda x: extract_structure(x.lower(),2))                                                                

In [4]:
# make struct dictionary
struct_dict = {}
for name_struct_i in first_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1
for name_struct_i in last_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1

In [5]:
struct_dict_keys = list(struct_dict.keys())
ethnic_series = ethnea_df['Ethnea'].str.lower()
pub_series = ethnea_df.PubCountry.str.lower()
ethnic_keys = list(np.unique(ethnic_series.values))
pub_keys = list(np.unique(pub_series.values))

In [6]:
len(pub_keys)

275

In [7]:
# load test train data
with open('train_test_full_index.pickle', 'rb') as f:
    trainIndex,testIndex,trainY,testY = pickle.load(f)

In [8]:
# transform the dataset into structure
def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

In [9]:
def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

#data_source = full_name_struct.apply(lambda x: transform_structure(x))

def generate_batch(first_name, last_name, i, batch_size=10000):
    len_name = len(first_name)
    #print(len_name)
    start = i*batch_size
    #print(start)
    end = start+batch_size if start+batch_size < len_name else len_name
    len_mat = end - start
    #first_name_ds_mat = np.zeros((len_mat,50),dtype=np.int32)
    #last_name_ds_mat = np.zeros((len_mat,50),dtype=np.int32)
    first_name_ds_mat = first_name[start:end]
    last_name_ds_mat = last_name[start:end]
    first_name_ds_mat = pad_sequences(first_name_ds_mat.apply(lambda x: transform_structure(x)),maxlen=50,value=0.)
    first_name_ds_mat = first_name_ds_mat.reshape(first_name_ds_mat.shape[0],1,first_name_ds_mat.shape[1])
    last_name_ds_mat = pad_sequences(last_name_ds_mat.apply(lambda x:transform_structure(x)),maxlen=50,value=0.)
    last_name_ds_mat = last_name_ds_mat.reshape(last_name_ds_mat.shape[0],1,last_name_ds_mat.shape[1])

    """
    for i in range(len_mat):
        x = first_name_struct.iloc[i]
        for y in x:
            first_name_ds_mat[i,struct_dict_keys.index(y)]+=1
        x = last_name_struct.iloc[i]
        for y in x:
            last_name_ds_mat[i,struct_dict_keys.index(y)]+=1
    """
    return first_name_ds_mat,last_name_ds_mat, range(start,end)

In [10]:
def transform_labels(x,my_keys):
    y = np.zeros(len(my_keys))
    y[my_keys.index(x)]=1
    return y


labels = np.array(list(map(lambda x: transform_labels(x,ethnic_keys),ethnic_series)))
#pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series)))

In [11]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tflearn.data_utils import to_categorical, pad_sequences



Using TensorFlow backend.


In [11]:
?Input

Object `Input` not found.


In [12]:
from keras.layers import Input
from keras.models import Model

"""
model = Sequential()
model.add(Embedding(len(name_struct_keys)+1,embedding_vector_length,input_length=max_sequence))
model.add(Conv1D(filters=embedding_vector_length,kernel_size=3,padding='same',activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(lstm_layer,dropout=0.8))
model.add(Bidirectional(LSTM(max_sequence*2,return_sequences=False),input_shape=(max_sequence,1)))
#model.add(TimeDistributed(keras.layers.Dense(len(ethnic_keys),activation='softmax')))
model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
#model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
"""

# max sequence length
seq_length = 100
# multi input with single output

# first name input
first_name_input = Input(shape=(1,50),name='first_name_input')
last_name_input = Input(shape=(1,50),name='last_name_input')

# pub input
pub_input = Input(shape=(len(pub_keys),),name='pub_input')

# first tensor for first name
first_name_l = Bidirectional(LSTM(1000,return_sequences=False))(first_name_input)
last_name_l = Bidirectional(LSTM(1000,return_sequences=False))(last_name_input)

# merge the two layer together
merge_first_last = keras.layers.concatenate([first_name_l,last_name_l])

# stack dense network for memory
full_conn_merge = Dense(1000, activation='relu')(merge_first_last)
x = keras.layers.concatenate([full_conn_merge,pub_input])
x = Dense(500, activation='relu')(x)
output_l = Dense(len(ethnic_keys),activation='softmax')(x)

model = Model(inputs=[first_name_input, last_name_input,pub_input], outputs=[output_l])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
first_name_input (InputLayer)    (None, 1, 50)         0                                            
____________________________________________________________________________________________________
last_name_input (InputLayer)     (None, 1, 50)         0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 2000)          8408000     first_name_input[0][0]           
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 2000)          8408000     last_name_input[0][0]            
___________________________________________________________________________________________

In [ ]:
model.summary()
first_trainX = first_name_struct[trainIndex]
first_testX = first_name_struct[testIndex]
last_trainX = last_name_struct[trainIndex]
last_testX = last_name_struct[testIndex]
trainY = labels[trainIndex]
testY = labels[testIndex]

#trainX =np.array([to_categorical(x,nb_classes=len(struct_dict_keys)+1) for x in trainX])
#testX =np.array([to_categorical(x,nb_classes=len(struct_dict_keys)+1) for x in testX])

mini_batch_size = 20000
len_mini_batch = round(len(trainY)/mini_batch_size)
batch_size = 5000

for x in range(1):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series[trainIndex].iloc[batch_range])))
        model.fit([y_first_trainX, y_last_trainX, pub_country],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
first_name_input (InputLayer)    (None, 1, 50)         0                                            
____________________________________________________________________________________________________
last_name_input (InputLayer)     (None, 1, 50)         0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 2000)          8408000     first_name_input[0][0]           
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 2000)          8408000     last_name_input[0][0]            
___________________________________________________________________________________________

20000/20000 [==============================] - 41s - loss: 0.6490 - acc: 0.8124     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6463 - acc: 0.8117     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.6535 - acc: 0.8108     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6348 - acc: 0.8160     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6517 - acc: 0.8105     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6406 - acc: 0.8106     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.6489 - acc: 0.8110     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6393 - acc: 0.8162     


In [ ]:
for x in range(5):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series[trainIndex].iloc[batch_range])))
        model.fit([y_first_trainX, y_last_trainX, pub_country],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

3547268
0
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6421 - acc: 0.8121     
3547268
20000
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.6257 - acc: 0.8164     
3547268
40000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.6192 - acc: 0.8194     
3547268
60000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.6041 - acc: 0.8232     
3547268
80000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.5815 - acc: 0.8320     
3547268
100000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.5622 - acc: 0.8379     
3547268
120000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.5573 - acc: 0.8390     
3547268
140000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.5233 - acc: 0.8497     
3547268
160000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.5252 - acc: 0.8494     
3547268
180000
Epoc

20000/20000 [==============================] - 42s - loss: 0.3179 - acc: 0.9066     
3547268
1740000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3170 - acc: 0.9093     
3547268
1760000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3101 - acc: 0.9122     
3547268
1780000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3113 - acc: 0.9103     
3547268
1800000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3181 - acc: 0.9092     
3547268
1820000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3212 - acc: 0.9082     
3547268
1840000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3245 - acc: 0.9073     
3547268
1860000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3015 - acc: 0.9134     
3547268
1880000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3120 - acc: 0.9109     
3547268
1900000
Epoch 1/1
2

Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3092 - acc: 0.9122     
3547268
3220000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3053 - acc: 0.9129     
3547268
3240000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2977 - acc: 0.9132     
3547268
3260000
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.3052 - acc: 0.9113     
3547268
3280000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3048 - acc: 0.9135     
3547268
3300000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3036 - acc: 0.9138     
3547268
3320000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.3116 - acc: 0.9109     
3547268
3340000
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.3135 - acc: 0.9087     
3547268
3360000
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.3167 - acc: 0.9107     
3547268
3380000
E

Epoch 1/1
20000/20000 [==============================] - 193s - loss: 0.3029 - acc: 0.9134    
3547268
2640000
Epoch 1/1
20000/20000 [==============================] - 194s - loss: 0.2898 - acc: 0.9158    
3547268
2660000
Epoch 1/1
20000/20000 [==============================] - 192s - loss: 0.2943 - acc: 0.9145    
3547268
2680000
Epoch 1/1
20000/20000 [==============================] - 196s - loss: 0.2955 - acc: 0.9114    
3547268
2700000
Epoch 1/1
20000/20000 [==============================] - 194s - loss: 0.2881 - acc: 0.9177    
3547268
2720000
Epoch 1/1
20000/20000 [==============================] - 194s - loss: 0.2886 - acc: 0.9148    
3547268
2740000
Epoch 1/1
20000/20000 [==============================] - 194s - loss: 0.2881 - acc: 0.9153    
3547268
2760000
Epoch 1/1
20000/20000 [==============================] - 194s - loss: 0.2830 - acc: 0.9190    
3547268
2780000
Epoch 1/1
20000/20000 [==============================] - 194s - loss: 0.2947 - acc: 0.9122    
3547268
2800000
E

Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2817 - acc: 0.9164     
3547268
2500000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2738 - acc: 0.9199     
3547268
2520000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2702 - acc: 0.9214     
3547268
2540000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2660 - acc: 0.9205     
3547268
2560000
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2837 - acc: 0.9185     
3547268
2580000
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2745 - acc: 0.9198     
3547268
2600000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2659 - acc: 0.9211     
3547268
2620000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2858 - acc: 0.9167     
3547268
2640000
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2753 - acc: 0.9195     
3547268
2660000
E

In [ ]:
for x in range(5):
    for y in range(len_mini_batch):
        y_first_trainX, y_last_trainX, batch_range =  generate_batch(first_trainX,last_trainX,y,mini_batch_size)
        pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys),pub_series[trainIndex].iloc[batch_range])))
        model.fit([y_first_trainX, y_last_trainX, pub_country],trainY[batch_range],epochs=1,batch_size=batch_size)
        #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)
        #print("Accuracy: %.2f%%" %(scores[1]*100))

Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2698 - acc: 0.9208     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2737 - acc: 0.9193     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2715 - acc: 0.9192     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2757 - acc: 0.9166     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2757 - acc: 0.9194     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2670 - acc: 0.9226     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2772 - acc: 0.9180     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2710 - acc: 0.9211     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2757 - acc: 0.9181     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2833 - acc: 0.9171     
Epoch 1/1
20000/20000 [===========================

20000/20000 [==============================] - 41s - loss: 0.2542 - acc: 0.9242     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2610 - acc: 0.9227     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2546 - acc: 0.9235     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2473 - acc: 0.9264     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2557 - acc: 0.9240     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2528 - acc: 0.9255     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2573 - acc: 0.9244     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2567 - acc: 0.9242     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2602 - acc: 0.9212     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2580 - acc: 0.9229     
Epoch 1/1
20000/20000 [==============================] - 41s

20000/20000 [==============================] - 41s - loss: 0.2313 - acc: 0.9310     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2325 - acc: 0.9308     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2307 - acc: 0.9317     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2371 - acc: 0.9280     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2364 - acc: 0.9281     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2241 - acc: 0.9321     
Epoch 1/1
20000/20000 [==============================] - 41s - loss: 0.2321 - acc: 0.9299     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2313 - acc: 0.9309     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2305 - acc: 0.9327     
Epoch 1/1
20000/20000 [==============================] - 42s - loss: 0.2382 - acc: 0.9286     
Epoch 1/1
20000/20000 [==============================] - 41s

In [62]:
def evaluate(firstX, lastX, testY, testIndex, mini_batch_size=20000):
    test_len = len(testY)
    len_mini_batch = round(test_len/mini_batch_size)
    scores = np.zeros(len_mini_batch)
    for i in range(len_mini_batch):
        y_first_trainX,y_last_trainX,batch_range = generate_batch(firstX,lastX,i,mini_batch_size)
        pub_country = np.array(list(map(lambda x: transform_labels(x,pub_keys), pub_series[testIndex].iloc[batch_range])))
        scores[i] = model.evaluate([y_first_trainX,y_last_trainX,pub_country],testY[batch_range],batch_size=2000)[1]
        print(scores[i])
    return np.average(scores)
    #scores = model.evaluate([first_testX, last_testX],testY,verbose=0)

In [57]:
evaluate(first_testX,last_testX, testY, testIndex)
#test
#y_first_trainX,y_last_trainX,batch_range = generate_batch(first_testX,last_testX,1,mini_batch_size)

886817
0
20000/20000 [==============================] - 30s    
0.81265
886817
20000
20000/20000 [==============================] - 30s    
0.81075
886817
40000
20000/20000 [==============================] - 30s    
0.812
886817
60000
19968/20000 [============================>.] - ETA: 0s0.81295
886817
80000
20000/20000 [==============================] - 30s     ETA: 
0.81
886817
100000
19968/20000 [============================>.] - ETA: 0s0.814
886817
120000
20000/20000 [==============================] - 30s    
0.8061
886817
140000
19968/20000 [============================>.] - ETA: 0s0.812
886817
160000
19968/20000 [============================>.] - ETA: 0s0.8087
886817
180000
19968/20000 [============================>.] - ETA: 0s0.81345
886817
200000
20000/20000 [==============================] - 30s    
0.8113
886817
220000
19968/20000 [============================>.] - ETA: 0s0.81225
886817
240000
19968/20000 [============================>.] - ETA: 0s0.8128
886817
260000
19968/20

0.81100795454545427

In [ ]:
evaluate(first_testX,last_testX, testY, testIndex)
#test
#y_first_trainX,y_last_trainX,batch_range = generate_batch(first_testX,last_testX,1,mini_batch_size)

20000/20000 [==============================] - 14s    
0.909500002861
20000/20000 [==============================] - 15s    
0.907749992609
20000/20000 [==============================] - 14s    
0.909549999237
20000/20000 [==============================] - 14s    
0.910799992085
20000/20000 [==============================] - 14s    
0.908799999952
20000/20000 [==============================] - 14s    
0.91060000658
20000/20000 [==============================] - 14s    
0.905499994755
20000/20000 [==============================] - 14s    
0.908800011873
20000/20000 [==============================] - 15s    
0.911449992657
20000/20000 [==============================] - 14s    
0.911999994516
20000/20000 [==============================] - 14s    
0.90894998908
20000/20000 [==============================] - 15s    
0.907499998808
20000/20000 [==============================] - 14s    
0.912050014734
20000/20000 [==============================] - 14s    
0.908299994469
20000/20000 [=========

In [50]:
#pub_series[testIndex].iloc[batch_range]
pub_series[testIndex].iloc[20000]

'china'

In [28]:
y_first_trainX.reshape(y_first_trainX.shape[0],1,y_first_trainX.shape[1]).shape

(10000, 1, 50)

In [39]:
def trans_name(name):
    name = name.lower()
    # transform space into underscore
    name = '_'+name.replace(' ','_')+'_'
    #transform the name into sequence structure
    ext_name = extract_structure(name)
    trans_name = transform_structure(ext_name)
    #name_ds_mat = np.zeros((1,len(struct_dict_keys)),dtype=np.int32)
    #for i,x in enumerate(trans_name):
    #    name_ds_mat[0,x-1]+=1    
    trans_name = pad_sequences([trans_name], maxlen=50,value=0.)
    trans_name = trans_name.reshape(trans_name.shape[0],1,trans_name.shape[1])

    return trans_name

def predict_ethnicity(fname,lname):
    # lower case the name
    fnamex = trans_name(fname)
    lnamex = trans_name(lname)
    pred = model.predict([np.array(fnamex),np.array(lnamex)])
    pred_class = np.argsort(pred[0])[::-1]
    return_item = []
    for x in np.argsort(pred[0])[::-1]:
        return_item.append((ethnic_keys[x],pred[0][x]))
    return return_item

In [12]:
predict_ethnicity('Hari','Cahyono')
#trans_name('Nikolaus')

NameError: name 'predict_ethnicity' is not defined

In [8]:
# embedd the structure vocabulary using text embedding and reduce the dimensionality

# convert the names into word structure vector
struct_dict_keys = list(struct_dict.keys())

def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

#data_source = full_name_struct.apply(lambda x: transform_structure(x))

In [9]:
def transform_labels(x):
    y = np.zeros(len(ethnic_keys))
    y[ethnic_keys.index(x)]=1
    return y

labels = np.array(list(map(lambda x: transform_labels(x),ethnic_series)))

In [26]:
# using tflearn make the graph creation simple
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split

# separate train and training set
trainX, testX, trainY, testY = train_test_split(data_source,[ethnic_keys.index(x) for x in ethnic_series],test_size = 0.2)

trainX = pad_sequences(trainX, maxlen=50,value=0.)
testX = pad_sequences(testX,maxlen=50,value=0.)
# Converting labels to binary vectors
trainY = to_categorical(trainY,nb_classes=len(ethnic_keys))
testY = to_categorical(testY,nb_classes=len(ethnic_keys))    

In [16]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import pickle

with open('train_test_fix.pickle', 'rb') as f:
    trainX,trainY,testX,testY,ethnic_keys,struct_dict_keys = pickle.load(f)
    #aha = pickle.load(f)

#with open('traintest-smote.pickle','rb') as f:
#    train_res,test_res = pickle.load(f)

with open('ethnic_keys.pickle','rb') as f:
    name_struct_keys,ethnic_keys = pickle.load(f)
        
embedding_vector_length = 1000
lstm_layer = 1000
max_sequence = 50

In [ ]:
# convert categorical to binary crossentropy
#trainY = np.array([np.where(x>0)[0][0] for x in trainY])
#testY = np.array([np.where(x>0)[0][0] for x in testY])

#test_res

In [2]:
model = Sequential()
model.add(Embedding(len(name_struct_keys)+1,embedding_vector_length,input_length=max_sequence))
model.add(Conv1D(filters=embedding_vector_length,kernel_size=3,padding='same',activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(lstm_layer,dropout=0.8))
model.add(Bidirectional(LSTM(max_sequence*2,return_sequences=False),input_shape=(max_sequence,1)))
#model.add(TimeDistributed(keras.layers.Dense(len(ethnic_keys),activation='softmax')))
model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
#model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
for x in range(10):
    model.fit(trainX,trainY,epochs=1,batch_size=1000)
    scores = model.evaluate(testX,testY,verbose=0)
    print("Accuracy: %.2f%%" %(scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 1000)          62696000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 1000)          3001000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               880800    
_________________________________________________________________
dense_1 (Dense)              (None, 23)                4623      
Total params: 66,582,423
Trainable params: 66,582,423
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
35653/35653 [==============================] - 778s - loss: 1.6879 - acc: 0.5265    
Accuracy: 75.88%
Epoch 1/1
35653/35653 [==============================] - 796s - loss: 0.5197 - acc: 0.8582    
Accuracy: 84.37%
Epoch 1/1
35653/35653 [==========

In [4]:
testX

array([[  112,  1331,  1332, ...,     0,     0,     0],
       [  251,   252,   232, ...,     0,     0,     0],
       [ 4633, 11731, 23123, ...,     0,     0,     0],
       ..., 
       [  696,   697,   698, ...,     0,     0,     0],
       [   25,    26,  5195, ...,     0,     0,     0],
       [ 8671,  8672,  6568, ...,     0,     0,     0]], dtype=int32)

In [12]:
from keras.models import model_from_json
from keras import backend as K

# compute the accuracy
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2
    #print(precision)

    # How many relevant items are selected?
    recall = c1 / c3
    #print(recall)

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def precision(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    return precision


def recall(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    recall = c1 / c3

    return recall

# load model
# load json and create model
json_file = open('model-keras-w.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#                               ,custom_objects= {'f1_score': f1_score})
loaded_model.load_weights("model-keras-embed-bilstm-womaxpool-10.h5")

loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy',f1_score,precision,recall])

In [18]:
scores = loaded_model.evaluate(testX,testY,verbose=0)

In [19]:
print('Accuracy: {}, F1: {}, Precision: {}, Recal: {}'.format(scores[1],scores[2],scores[3],scores[4]))

Accuracy: 0.8561812878216707, F1: 0.8601400889915978, Precision: 0.8757469038779302, Recal: 0.845523895023381
